In [1]:
pip install openai==0.28

^C
Note: you may need to restart the kernel to use updated packages.


In [3]:
import openai

In [4]:
openai.api_key=""

In [5]:
system_message = """Your name is Tom and you are now communicating with a psychologist in a therapy session.
Answer the following questions as Tom and respond in a casual tone. Each of your replies should be no longer than three sentences."""

In [ ]:
#Upload files

In [ ]:
training_file_name = 'SFT_dataset/GPT/SFT_GPT_362_oneturn_train.jsonl'
validation_file_name = 'SFT_dataset/GPT/SFT_GPT_362_oneturn_val.jsonl'

In [ ]:
with open(training_file_name, "rb") as file:
    traning_response = openai.File.create(file=file, purpose="fine-tune")

training_file_id = traning_response["id"]


with open(validation_file_name, "rb") as file:
    validation_response = openai.File.create(file=file, purpose="fine-tune")

validation_file_id = validation_response["id"]

print("training file id:", training_file_id)
print("validation file id:", validation_file_id)

training_file_name = 'SFT_dataset/GPT/SFT_GPT_362_oneturn_train.jsonl'
validation_file_name = 'SFT_dataset/GPT/SFT_GPT_362_oneturn_val.jsonl'
training file id: file-NxCZBl0clKMFBjHyUmqs7QzG
validation file id: file-EJuJ2rTrl476MiXs58zmtCJf

In [ ]:
#Fine tuning

In [ ]:
suffix_name = "gpt-4-oneturn"

response = openai.FineTuningJob.create(
    training_file = training_file_id,
    validation_file = validation_file_id,
    model = "gpt-4",
    suffix = suffix_name,
    hyperparameters={
    "n_epochs":2
  }
)

job_id = response["id"]

In [ ]:
response = openai.FineTuningJob.retrieve(job_id)
print(response)

gpt-3.5-oneturn   "fine_tuned_model": "ft:gpt-3.5-turbo-0125:the-university-of-melbourne:gpt-3-5-oneturn:9PUQsR99"

In [ ]:
response = openai.FineTuningJob.list_events(id=job_id, limit=1574)

events = response["data"]
events.reverse()

for event in events:
    print (event["message"])

In [ ]:
import csv

response = openai.FineTuningJob.list_events(id=job_id, limit=1574)
events = response["data"]
events.reverse()

with open('events.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Step', 'Full Validation Loss'])

    for event in events:
        message = event["message"]
        if "validation loss" in message:
            step = message.split(': ')[0].split(' ')[1]
            loss = message.split('validation loss=')[1]
            writer.writerow([step, loss])

In [ ]:
fine_tuned_model_id = response["fine_tuned_model"]

In [ ]:
fine_tuned_model_id = 'ft:gpt-3.5-turbo-0125:the-university-of-melbourne:gpt-3-5-oneturn:9PUQsR99'

In [ ]:
#Test

In [ ]:
# Read questions from question.txt file and extract them
questions = []
with open("question.txt", "r") as file:
    lines = file.readlines()
    for line in lines:
        # Extract questions after the question number
        question = line.split(".", 1)[-1].strip()
        questions.append(question)

In [ ]:
# Loop for dialogue inference
with open("gpt_finetuned.txt", "a") as output_file:
    for question in questions:
        
        test_messages=[]
        test_messages.append({"role": "system", "content": system_message})
        user_message = question
        test_messages.append({"role": "user", "content": user_message})

        #Response after fine-tuning
        response = openai.ChatCompletion.create(
            model = fine_tuned_model_id, messages = test_messages, temperature = 1, max_tokens=500, frequency_penalty=1
        )
        assistant_response = response["choices"][0]["message"]["content"]
        print(response["choices"][0]["message"]["content"])
        
        # Append each answer to gpt_finetuned.txt
        output_file.write(assistant_response + "\n")

In [ ]:
test_messages=[]
test_messages.append({"role": "system", "content": system_message})
user_message = "How do you feel recently?"
test_messages.append({"role": "user", "content": user_message})

In [ ]:
!pip install --upgrade openai

In [7]:
from openai import OpenAI
client = OpenAI()

response = client.embeddings.create(
    input="Your text string goes here",
    model="text-embedding-3-small"
)

print(response.data[0].embedding)

ImportError: cannot import name 'OpenAI' from 'openai' (D:\Download\anaconda3\lib\site-packages\openai\__init__.py)

In [ ]:
#Response after fine-tuning
response = openai.Completion.create(
    model = "text-embedding-ada-002", messages = test_messages, temperature = 0.5, max_tokens=500, frequency_penalty=0.5
)
print(response["choices"][0]["message"]["content"])

In [ ]:
#Response without fine-tuning
response = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo", messages = test_messages, temperature = 0, max_tokens=500
)
print(response["choices"][0]["message"]["content"])

In [ ]:
# Loop for dialogue inference
with open("gpt.txt", "a") as output_file:
    for question in questions:
        
        test_messages=[]
        test_messages.append({"role": "system", "content": system_message})
        user_message = question
        test_messages.append({"role": "user", "content": user_message})

        #Response without fine-tuning
        response = openai.ChatCompletion.create(
            model = "gpt-3.5-turbo", messages = test_messages, temperature = 0, max_tokens=500
        )
        
        assistant_response = response["choices"][0]["message"]["content"]
        print(response["choices"][0]["message"]["content"])
        
        # Append each answer to gpt_finetuned.txt
        output_file.write(assistant_response + "\n")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.1.6 requires openai<2.0.0,>=1.24.0, but you have openai 0.28.0 which is incompatible.


  Using cached openai-0.28.0-py3-none-any.whl (76 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.30.1
    Uninstalling openai-1.30.1:
      Successfully uninstalled openai-1.30.1
